<a href="https://colab.research.google.com/github/sbj6364/voice-converter/blob/main/04_resynthetic1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4번 : 음성 데이터 재합성하기 - 1

주어진 음원에서 STFT를 추출하고, 이로부터 다시 복원된 wave를 생성해서 저장하는 코드를 작성해주세요.


#### 베이스라인
~~~python
audio, sr = librosa.load('speech.wav', sr=None)
spectrogram = librosa.core.stft(audio)

audio_recon = ''' code here '''

sf.write('speech_recon.wav', audio_recon, sr)
~~~

#### 설명
① 앞선 일련의 예제를 통해서 우리는 주어진 음성으로 부터 여러 feature를 '추출' 하는 방법을 익혔습니다. 하지만 우리의 최종 목표는 변형된 음성을 '생성'해내는 모듈을 디자인하는 것 입니다.

② 일반적으로 목소리를 변형시키는 과정은 아래와 같이 생각해볼 수 있습니다.

`음성 —> (1) feature 추출 —> (2) feature 변형 —> (3) feature 재합성 —> 변형된 음성`

현재까지 우리는 (1)에 해당하는 과정을 진행했으며, 성공적인 모듈 디자인을 위해서는 (2) 와 (3) 에 대해서 파악해야 합니다.

(1) 추출과 (3) 재합성이 가능하기 위해서는 추출 연산의 역과정이 정의될 수 있어야 합니다. 대표적으로 Short time fourier transform은 inver short time fourier transform을 정의할 수 있는 변형이기 때문에 reconstruction이 가능합니다.



In [1]:
import IPython.display as ipd
ipd.Audio('speech.wav')

In [2]:
import numpy as np  
import librosa
import matplotlib 
matplotlib.use("Agg")
import matplotlib.pyplot as plt 
import librosa.display

In [3]:
def Spectrogram(wav):
		stft = librosa.stft(wav)
		stft = np.abs(stft)
		return stft

In [4]:
audio, sr = librosa.load('speech.wav', sr=None)
spectrogram = np.log(Spectrogram(audio)+1e-5)

plt.figure(figsize=(16,9))

plt.subplot(2,1,1)
plt.plot(audio)
plt.xlim([0,len(audio)])
plt.title('waveform')

plt.subplot(2,1,2)
librosa.display.specshow(spectrogram)
plt.title('spectrogram')

plt.tight_layout()
plt.savefig('example1_output.png')
# plt.close()

In [5]:
!pip install pyworld

     |████████████████████████████████| 215kB 6.5MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pyworld: filename=pyworld-0.3.0-cp37-cp37m-linux_x86_64.whl size=609276 sha256=3067ca5ee30e064c7ad4dccce2baff352de6a2204c1bc96ba52d6b27e731eeeb
  Stored in directory: /root/.cache/pip/wheels/dc/27/35/dc77501124e514e06bd33bc41253b61dbaea19599c6da2679b
Successfully built pyworld


In [6]:
import pyworld as pw 

audio, sr = librosa.load('speech.wav', sr=None)
spectrogram = np.log(np.abs(librosa.core.stft(audio))+1e-5)

audio = np.asarray(audio, dtype='float64')

_f0, t = pw.dio(audio, sr)

f0 = pw.stonemask(audio, _f0, t, sr)

rms = librosa.feature.rms(audio)

plt.figure(figsize=(16,9))

plt.subplot(4,1,1)
plt.plot(audio)
plt.xlim([0,len(audio)])
plt.title('waveform')

plt.subplot(4,1,2)
librosa.display.specshow(spectrogram)
plt.title('spectrogram')

plt.subplot(4,1,3)
plt.plot(rms[0])
plt.xlim([0,len(rms[0])])
plt.title('rms')

plt.subplot(4,1,4)
plt.plot(f0)
plt.xlim([0,len(f0)])
plt.title('pitch')

plt.tight_layout()
plt.savefig('example2_output.png')
plt.close()

In [7]:
import soundfile as sf

audio, sr = librosa.load('speech.wav', sr=None)
onsets = librosa.onset.onset_detect(audio, sr=44100, hop_length=512)
onsets = librosa.frames_to_time(onsets, sr=44100, hop_length=512)
# print(onset_times) # 확인용 

plt.figure(figsize=(16,9))
plt.subplot(2,1,1)
plt.plot(audio)
plt.xlim([0,len(audio)])

for item in onsets:
    plt.axvline(x=(int)(item*sr), c='r')
    # print(item) # 확인용
    # 세로 선을 x 위치에 그려주는 함수. 
plt.title('waveform')


plt.subplot(2,1,2)
librosa.display.specshow(spectrogram)
for item in onsets:
    plt.axvline(x=(int)(item*sr)/512, c='r')
plt.title('spectrogram')

plt.tight_layout()
plt.savefig('example3_output.png')
plt.close()

for i in range(len(onsets[:-1])):
    sf.write('example3_output_'+str(i).zfill(2)+'.wav', audio[(int)(onsets[i]*sr):(int)(onsets[i+1]*sr)], sr)

In [50]:
ipd.Audio('/content/example3_output_01.wav')

In [9]:
ipd.Audio('/content/example3_output_02.wav')

In [10]:
audio, sr = librosa.load('speech.wav', sr=None)
spectrogram = librosa.core.stft(audio)
 
audio_recon = librosa.istft(spectrogram, hop_length=512)

sf.write('speech_recon.wav', audio_recon, sr)

In [11]:
ipd.Audio('/content/speech_recon.wav')